In [1]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point, MultiPoint # To create line geometries that can be used in a GeoData
import numpy as np
import math
from math import floor
from osgeo import gdal, ogr
import struct
from shapely.ops import unary_union
import rasterio
from scipy import stats

In [2]:
FORECAST = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Artikler\ExpScore\avalanche-bulletin_2017-12-01--2024-06-01.csv'
SHAPEFILE = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\GPX tracks - Strava\clean_data\carepanel\carepanel_tracks_final_duplicates_removed_2023_2024.shp'
RUN_DISTANCE = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\Flow-Py - Norge\alpha_23\runout_distance.tif'
ExpScore_PRA = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\Flow-Py - Norge\pra.tif'
ELEVATION = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\DTM10\dtm10_apr_int16.tif'
SLOPE = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\DTM10\dtm10_apr_slope_int16.tif'
REGION = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\Varslingsregioner\Varslingsregioner_1km.tif'
ASPECT = r'C:\Users\htla\OneDrive - Norges vassdrags- og energidirektorat\Datasett (GIS)\DTM10\dtm10_apr_cardinal_directions.tif'

In [3]:
forecast = pd.read_csv(FORECAST)
gdf = gpd.read_file(SHAPEFILE)

In [1]:
# gdf.head()

In [5]:
gdf['datetime'] = gdf['date'] + gdf['time']

In [7]:
forecast['Dato'] = pd.to_datetime(forecast['Dato'], format='%Y-%m-%d').dt.date
gdf['datetime'] = pd.to_datetime(gdf['datetime'], format='%Y-%m-%d%H_%M_%S').dt.date

In [2]:
# gdf

In [9]:
# functions
def extract_string(input_string, datatype):
    # Split the string into a list of numbers and remove '\n'
    string = input_string.strip("[]").replace('\n', '').split()
    
    # Convert the list of strings to a NumPy array of integers
    string = np.array(string, dtype=datatype)
    return string

def get_exposed_sector(SP_asp, SP_min, SP_max, elevation, aspect, runout, pra):
    # Correct aspect dictionary according to specified format
    aspect_dict = {'N': 1, 'NE': 2, 'E': 3, 'SE': 4, 'S': 5, 'SW': 6, 'W': 7, 'NW': 8}

    # Filter aspects based on SP1_asp
    valid_aspects = [aspect_dict[asp] for asp in SP_asp.split(', ')]

    # Filter based on corrected aspect values and updated elevation criteria
    valid_indices = (np.isin(aspect, valid_aspects) & (elevation >= SP_min) & (elevation <= SP_max))
    
    # Count of valid values
    num_valid_values = np.sum(valid_indices)

    # Calculate sums for valid indices
    count_runout = np.sum(runout[valid_indices] > 0)
    count_pra = np.sum(pra[valid_indices] > 0)
    sum_runout = np.sum(runout[valid_indices])
    sum_pra = np.sum(pra[valid_indices])
    
    return sum_runout, sum_pra, num_valid_values, count_runout, count_pra

def get_exposed_sector_two(SP1_asp, SP1_min, SP1_max, SP2_asp, SP2_min, SP2_max, elevation, aspect, runout, pra):
    # Correct aspect dictionary according to specified format
    aspect_dict = {'N': 1, 'NE': 2, 'E': 3, 'SE': 4, 'S': 5, 'SW': 6, 'W': 7, 'NW': 8}

    # Filter aspects based on SP1_asp
    valid_aspects1 = [aspect_dict[asp] for asp in SP1_asp.split(', ')]

    # Filter based on corrected aspect values and updated elevation criteria
    valid_indices1 = (np.isin(aspect, valid_aspects1) & (elevation >= SP1_min) & (elevation <= SP1_max))
    
    # Filter aspects based on SP2_asp
    valid_aspects2 = [aspect_dict[asp] for asp in SP2_asp.split(', ')]

    # Filter based on corrected aspect values and updated elevation criteria
    valid_indices2 = (np.isin(aspect, valid_aspects2) & (elevation >= SP2_min) & (elevation <= SP2_max))
    
    valid_indices = valid_indices1 | valid_indices2
    
    # Count of valid values
    num_valid_values = np.sum(valid_indices)

    # Calculate sums for valid indices
    count_runout = np.sum(runout[valid_indices] > 0)
    count_pra = np.sum(pra[valid_indices] > 0)
    sum_runout = np.sum(runout[valid_indices])
    sum_pra = np.sum(pra[valid_indices])
    
    return sum_runout, sum_pra, num_valid_values, count_runout, count_pra

def get_exposed_sector_three(SP1_asp, SP1_min, SP1_max, SP2_asp, SP2_min, SP2_max, SP3_asp, SP3_min, SP3_max, elevation, aspect, runout, pra):
    # Correct aspect dictionary according to specified format
    aspect_dict = {'N': 1, 'NE': 2, 'E': 3, 'SE': 4, 'S': 5, 'SW': 6, 'W': 7, 'NW': 8}

    # Filter aspects based on SP1_asp
    valid_aspects1 = [aspect_dict[asp] for asp in SP1_asp.split(', ')]

    # Filter based on corrected aspect values and updated elevation criteria
    valid_indices1 = (np.isin(aspect, valid_aspects1) & (elevation >= SP1_min) & (elevation <= SP1_max))
    
    # Filter aspects based on SP2_asp
    valid_aspects2 = [aspect_dict[asp] for asp in SP2_asp.split(', ')]

    # Filter based on corrected aspect values and updated elevation criteria
    valid_indices2 = (np.isin(aspect, valid_aspects2) & (elevation >= SP2_min) & (elevation <= SP2_max))
    
    # Filter aspects based on SP2_asp
    valid_aspects3 = [aspect_dict[asp] for asp in SP3_asp.split(', ')]

    # Filter based on corrected aspect values and updated elevation criteria
    valid_indices3 = (np.isin(aspect, valid_aspects3) & (elevation >= SP3_min) & (elevation <= SP3_max))
    
    valid_indices = valid_indices1 | valid_indices2 | valid_indices3
    
    # Count of valid values
    num_valid_values = np.sum(valid_indices)

    # Calculate sums for valid indices
    count_runout = np.sum(runout[valid_indices] > 0)
    count_pra = np.sum(pra[valid_indices] > 0)
    sum_runout = np.sum(runout[valid_indices])
    sum_pra = np.sum(pra[valid_indices])
    
    return sum_runout, sum_pra, num_valid_values, count_runout, count_pra

def identify_aspects(SP1_asp, SP2_asp, SP3_asp):
    # Define the possible inputs
    possible_inputs = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

    # Initialize a set to store unique valid cardinal directions
    unique_cardinal_directions = set()

    # Collect all input strings into a list
    input_strings = [SP1_asp, SP2_asp, SP3_asp]

    # Iterate through each input string and add valid directions to the set
    for input_str in input_strings:
        if not pd.isna(input_str):
            directions = input_str.split(', ')
            for direction in directions:
                if direction in possible_inputs:
                    unique_cardinal_directions.add(direction)

    # Create a comma-separated string from the unique valid cardinal directions
    output_str = ', '.join(sorted(unique_cardinal_directions))

    # Print the final result
    return output_str

def find_minimum(SP1_min, SP2_min, SP3_min):
    # Create a list to store the values of SP1_min, SP2_min, and SP3_min
    values = [SP1_min, SP2_min, SP3_min]

    # Filter out None or empty values and find the minimum
    min_value = min(filter(lambda x: x is not None and x != '', values))

    return min_value

def find_maximum(SP1_max, SP2_max, SP3_max):
    # Create a list to store the values of SP1_min, SP2_min, and SP3_min
    values = [SP1_max, SP2_max, SP3_max]

    # Filter out None or empty values and find the minimum
    max_value = max(filter(lambda x: x is not None and x != '', values))

    return max_value

def extract_values_from_raster(src_filename, line_gdf):
    src_ds=gdal.Open(src_filename) 
    gt_forward=src_ds.GetGeoTransform()
    gt_reverse=gdal.InvGeoTransform(gt_forward)
    rb=src_ds.GetRasterBand(1)

    ds=ogr.Open(line_gdf['geometry'].to_json())
    lyr=ds.GetLayer()

    li_values = list()
    for feat in lyr:
        geom = feat.GetGeometryRef()
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        px, py = gdal.ApplyGeoTransform(gt_reverse, mx, my)
        px = floor(px) #x pixel
        py = floor(py) #y pixel

        structval=rb.ReadRaster(px,py,1,1,buf_type=gdal.GDT_UInt16) #Assumes 16 bit int aka 'short'
        intval = struct.unpack('h' , structval) # use the 'short' format code (2 bytes) not int (4 bytes)
        li_values.append(intval[0])
        arr = np.array(li_values)
    
    return arr

In [3]:
# len(gdf)

In [4]:
%%time

acc_pra = 0.75
acc_ro = 0.25
trip_pra = 0.1794
trip_ro = 0.8206
rr = (acc_ro/trip_ro)/(acc_pra/trip_pra)

data = pd.DataFrame()

for i in range(0, 4202, 1):
    print(i)
    track = gdf['geometry'][i]
    row = gdf[['filename', 'user', 'id', 'datetime']].iloc[i:i+1]
    date = row.iloc[0]['datetime']
    
    # Generate points for every 10 meter along line
    distance_delta = 10
    distances = np.arange(0, track.length, distance_delta)
    points = [track.interpolate(distance) for distance in distances] + [track.boundary[1]]
    multipoint = unary_union(points)
    line_df = pd.DataFrame()
    
    line_gdf = gpd.GeoDataFrame(line_df, geometry=[multipoint,])
    line_gdf = line_gdf.explode().reset_index(level=0).drop(['level_0'], axis=1)
    
    # Extract ExpScale_Runout
    run = extract_values_from_raster(RUN_DISTANCE, line_gdf)
    
    run[np.where(run == 10000)] = 0
    lam = 0.01639453
    alpha=0.8153966
    run = np.exp(-(lam*run)**alpha)
    run[np.where(run == 1)] = 0
    row['ExpScore_Runout_data'] = [run]
    row['ExpScore_Runout'] = run.sum()
    
    # Extract ExpScore_PRA
    pra = extract_values_from_raster(ExpScore_PRA, line_gdf)
    pra[np.where(pra == 128)] = 0
    pra = pra/100
    new_min = rr
    new_max = 1
    renormalized_pra = np.where(pra == 0, 0, new_min + (new_max - new_min) * pra)
    row['ExpScore_PRA_data'] = [renormalized_pra]
    row['ExpScore_PRA'] = renormalized_pra.sum()
    
    # row['ExpScore_PRA_data'] = [pra]
    # row['ExpScore_PRA'] = pra.sum()
    
    # row['ExpScore'] = (run.sum()*0.072857113)+pra.sum()
    # row['ExpScore'] = (run.sum()*0.072)+pra.sum()
    
    # Extract ELEVATION
    ele = extract_values_from_raster(ELEVATION, line_gdf)
    # pra[np.where(pra == 128)] = 0
    row['elevation'] = [ele]
    row['elevation_maks'] = ele.max()
    row['elevation_min'] = ele.min()
    
    # Extract SLOPE
    slo = extract_values_from_raster(SLOPE, line_gdf)
    # pra[np.where(pra == 128)] = 0
    row['slope'] = [slo]
    
    # Extract REGION
    reg = extract_values_from_raster(REGION, line_gdf)
    # pra[np.where(pra == 128)] = 0
    row['regionID'] = stats.mode(reg)[0][0]
    
    # Extract ASPECT
    asp = extract_values_from_raster(ASPECT, line_gdf)
    # pra[np.where(pra == 128)] = 0
    row['aspect'] = [asp]
    row['aspect_mode'] = stats.mode(asp)[0][0]
    row['length'] = len(asp)*10
    
    # Filter forecast
    region = int(row.iloc[0].regionID)
    filtered_forecast = forecast[(forecast['Dato'] == date) & (forecast['Region-ID'] == region)]
    filtered_forecast = filtered_forecast.fillna(np.nan)
    
    # Check if the DataFrame is empty
    if filtered_forecast.empty:
        continue
        
    else:
        row['danger_level'] = filtered_forecast.iloc[0].Faregrad
        row['SP1'] = filtered_forecast.iloc[0].SP1
        row['SP1_size'] = filtered_forecast['Skredstørrelse SP1'].iloc[0]
        row['SP1_distribution'] = filtered_forecast['Utbredelse SP1'].iloc[0]
        row['SP1_sensitivity'] = filtered_forecast['Utløsbarhet SP1'].iloc[0]  

        row['SP2'] = filtered_forecast.iloc[0].SP2
        row['SP2_size'] = filtered_forecast['Skredstørrelse SP2'].iloc[0]
        row['SP2_distribution'] = filtered_forecast['Utbredelse SP2'].iloc[0]
        row['SP2_sensitivity'] = filtered_forecast['Utløsbarhet SP2'].iloc[0]

        row['SP3'] = filtered_forecast.iloc[0].SP3
        row['SP3_size'] = filtered_forecast['Skredstørrelse SP3'].iloc[0]
        row['SP3_distribution'] = filtered_forecast['Utbredelse SP3'].iloc[0]
        row['SP3_sensitivity'] = filtered_forecast['Utløsbarhet SP3'].iloc[0]
        
        # Extract data from string
        row.iloc[0].elevation
        row.iloc[0].aspect
        row.iloc[0].ExpScore_PRA_data
        row.iloc[0].ExpScore_Runout_data
        
        SP1_asp = []
        SP1_min = []
        SP1_max = []        
        SP2_asp = []
        SP2_min = []
        SP2_max = []
        SP3_asp = []
        SP3_min = []
        SP3_max = []

        # SP1
        if pd.isna(filtered_forecast.iloc[0, 8]) == True:
            row['ExpScore_SP1_runout'] = -1
            row['ExpScore_SP1_pra'] = -1
            row['SP1_num_exposed_cells'] = -1
            row['SP1_asp'] = -1
            row['SP1_min'] = -1
            row['SP1_max'] = -1
            row['SP1_num_exposed_cells_runout'] = -1
            row['SP1_num_exposed_cells_pra'] = -1
        else:
            row['SP1_asp'] = filtered_forecast.iloc[0, 8]
            row['SP1_min'] = filtered_forecast.iloc[0, 9]
            row['SP1_max'] = filtered_forecast.iloc[0, 10]
            row['ExpScore_SP1_runout'], row['ExpScore_SP1_pra'], row['SP1_num_exposed_cells'], row['SP1_num_exposed_cells_runout'], row['SP1_num_exposed_cells_pra'] = get_exposed_sector(row.iloc[0].SP1_asp, row.iloc[0].SP1_min, row.iloc[0].SP1_max, row.iloc[0].elevation, row.iloc[0].aspect, row.iloc[0].ExpScore_Runout_data, row.iloc[0].ExpScore_PRA_data)

        # SP2
        if pd.isna(filtered_forecast.iloc[0, 15]) == True:
            row['ExpScore_SP2_runout'] = -1
            row['ExpScore_SP2_pra'] = -1
            row['SP2_num_exposed_cells'] = -1
            row['SP2_asp'] = -1
            row['SP2_min'] = -1
            row['SP2_max'] = -1
            row['SP2_num_exposed_cells_runout'] = -1
            row['SP2_num_exposed_cells_pra'] = -1
            row['SP1_2_num_exposed_cells_runout'] = -1
            row['SP1_2_num_exposed_cells_pra'] = -1
            row['SP1_2_num_exposed_cells'] = -1
            row['ExpScore_SP1_2_runout'] = -1
            row['ExpScore_SP1_2_pra'] = -1
        else:
            row['SP2_asp'] = filtered_forecast.iloc[0, 15]
            row['SP2_min'] = filtered_forecast.iloc[0, 16]
            row['SP2_max'] = filtered_forecast.iloc[0, 17]
            row['ExpScore_SP2_runout'], row['ExpScore_SP2_pra'], row['SP2_num_exposed_cells'], row['SP2_num_exposed_cells_runout'], row['SP2_num_exposed_cells_pra'] = get_exposed_sector(row.iloc[0].SP2_asp, row.iloc[0].SP2_min, row.iloc[0].SP2_max, row.iloc[0].elevation, row.iloc[0].aspect, row.iloc[0].ExpScore_Runout_data, row.iloc[0].ExpScore_PRA_data)
            row['ExpScore_SP1_2_runout'], row['ExpScore_SP1_2_pra'], row['SP1_2_num_exposed_cells'], row['SP1_2_num_exposed_cells_runout'], row['SP1_2_num_exposed_cells_pra'] = get_exposed_sector_two(row.iloc[0].SP1_asp, row.iloc[0].SP1_min, row.iloc[0].SP1_max, row.iloc[0].SP2_asp, row.iloc[0].SP2_min, row.iloc[0].SP2_max, row.iloc[0].elevation, row.iloc[0].aspect, row.iloc[0].ExpScore_Runout_data, row.iloc[0].ExpScore_PRA_data)
        # SP3
        if pd.isna(filtered_forecast.iloc[0, 22]) == True:
            row['ExpScore_SP3_runout'] = -1
            row['ExpScore_SP3_pra'] = -1
            row['SP3_num_exposed_cells'] = -1
            row['SP3_asp'] = -1
            row['SP3_min'] = -1
            row['SP3_max'] = -1
            row['SP3_num_exposed_cells_runout'] = -1
            row['SP3_num_exposed_cells_pra'] = -1
            row['SP1_3_num_exposed_cells_runout'] = -1
            row['SP1_3_num_exposed_cells_pra'] = -1
            row['SP1_3_num_exposed_cells'] = -1
            row['ExpScore_SP1_3_runout'] = -1
            row['ExpScore_SP1_3_pra'] = -1
        else:
            row['SP3_asp'] = filtered_forecast.iloc[0, 22]
            row['SP3_min'] = filtered_forecast.iloc[0, 23]
            row['SP3_max'] = filtered_forecast.iloc[0, 24]
            row['ExpScore_SP3_runout'], row['ExpScore_SP3_pra'], row['SP3_num_exposed_cells'], row['SP3_num_exposed_cells_runout'], row['SP3_num_exposed_cells_pra'] = get_exposed_sector(row.iloc[0].SP3_asp, row.iloc[0].SP3_min, row.iloc[0].SP3_max, row.iloc[0].elevation, row.iloc[0].aspect, row.iloc[0].ExpScore_Runout_data, row.iloc[0].ExpScore_PRA_data)
            row['ExpScore_SP1_3_runout'], row['ExpScore_SP1_3_pra'], row['SP1_3_num_exposed_cells'], row['SP1_3_num_exposed_cells_runout'], row['SP1_3_num_exposed_cells_pra'] = get_exposed_sector_three(row.iloc[0].SP1_asp, row.iloc[0].SP1_min, row.iloc[0].SP1_max, row.iloc[0].SP2_asp, row.iloc[0].SP2_min, row.iloc[0].SP2_max, row.iloc[0].SP3_asp, row.iloc[0].SP3_min, row.iloc[0].SP3_max, row.iloc[0].elevation, row.iloc[0].aspect, row.iloc[0].ExpScore_Runout_data, row.iloc[0].ExpScore_PRA_data)
            
        # print(row)
        columns_to_drop = ['elevation', 'slope', 'aspect', 'ExpScore_PRA_data', 'ExpScore_Runout_data']  # Replace with the names of the columns you want to drop
        row = row.drop(columns=columns_to_drop)
        
    data = data.append(row)

NameError: name 'pd' is not defined

3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


In [12]:
data.to_csv('ExpScore.csv')